In [1]:
import numpy as np
import pandas as pd
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from model import *
import torch.optim as optim
import torch.utils.data as Data

# use parser for implementation 
import argparse
from train import *

In [2]:
dataframe = pd.read_csv("Datasets/train_set_en.csv")

In [3]:
dataframe

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,...,ATGC_norm,HTGD_norm,ATGD_norm,DiffPts_norm,DiffFormPts_norm,HTP_norm,ATP_norm,FTR_OHE1,FTR_OHE2,FTR_OHE3
0,2009-08-15,Aston Villa,Wigan,0,2,0,0,1,A,11,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0
1,2009-08-15,Blackburn,Man City,0,2,0,0,1,A,17,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0
2,2009-08-15,Bolton,Sunderland,0,1,0,0,1,A,11,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0
3,2009-08-15,Chelsea,Hull,2,1,2,1,1,D,26,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.0
4,2009-08-15,Everton,Arsenal,1,6,0,0,3,A,8,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,2019-05-12,Leicester,Chelsea,0,0,1,0,0,D,9,...,0.481481,0.041667,0.347826,-0.338983,-0.076923,0.459459,0.639640,0.0,1.0,0.0
3796,2019-05-12,Liverpool,Wolves,2,0,2,1,0,H,13,...,0.543210,0.902778,0.043478,0.627119,0.384615,0.846847,0.513514,0.0,0.0,1.0
3797,2019-05-12,Man United,Cardiff,0,2,0,0,1,A,26,...,0.851852,0.180556,-0.536232,0.593220,0.153846,0.594595,0.279279,1.0,0.0,0.0
3798,2019-05-12,Southampton,Huddersfield,1,1,1,1,0,H,10,...,0.925926,-0.277778,-0.782609,0.389831,0.307692,0.342342,0.135135,0.0,1.0,0.0


In [4]:
data_dic = create_data_for_each_team(dataframe)

In [5]:
data_dic

{'Aston Villa':            Date     HomeTeam     AwayTeam FTHG FTAG FTR HTHG HTAG HTR  HS  \
 0    2009-08-15  Aston Villa        Wigan    0    2   0    0    1   A  11   
 1    2009-08-24    Liverpool  Aston Villa    1    3   0    0    2   A  21   
 2    2009-08-30  Aston Villa       Fulham    2    0   2    1    0   H   9   
 3    2009-09-13   Birmingham  Aston Villa    0    1   0    0    0   D  11   
 4    2009-09-19  Aston Villa   Portsmouth    2    0   2    2    0   H   7   
 ..          ...          ...          ...  ...  ...  ..  ...  ...  ..  ..   
 261  2016-04-16   Man United  Aston Villa    1    0   2    1    0   H  13   
 262  2016-04-16  Aston Villa      Chelsea    0    4   0    0    2   A  15   
 263  2016-04-16  Aston Villa  Bournemouth    1    2   0    0    1   A   6   
 264  2016-05-16      Arsenal  Aston Villa    4    0   2    1    0   H  16   
 265  2016-05-16  Aston Villa    Newcastle    0    0   1    0    0   D   2   
 
      ... ATGC_norm HTGD_norm ATGD_norm DiffPts

In [11]:
team_series_data, team_series_label = create_dataset(data_dic, 5)
print(team_series_data.shape)
print(team_series_label.shape)

torch.Size([524440, 5, 28])
torch.Size([524440])


In [12]:
team_series_data

tensor([[[ 2.0090e+03,  6.0000e+00,  1.1000e+01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.0090e+03,  6.0000e+00,  1.1000e+01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.0090e+03,  1.1000e+01,  2.0000e+01,  ...,  5.0000e-01,
           1.0000e+00,  0.0000e+00],
         [ 2.0090e+03,  1.1000e+01,  1.0000e+00,  ...,  0.0000e+00,
           1.0000e+00,  1.0000e+00],
         [ 2.0090e+03,  2.0000e+00,  6.0000e+00,  ...,  5.0000e-01,
           5.0000e-01,  0.0000e+00]],

        [[ 2.0090e+03,  6.0000e+00,  1.1000e+01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 2.0090e+03,  1.1000e+01,  2.0000e+01,  ...,  5.0000e-01,
           1.0000e+00,  0.0000e+00],
         [ 2.0090e+03,  1.1000e+01,  1.0000e+00,  ...,  0.0000e+00,
           1.0000e+00,  1.0000e+00],
         [ 2.0090e+03,  2.0000e+00,  6.0000e+00,  ...,  5.0000e-01,
           5.0000e-01,  0.0000e+00],
         [ 2.0090e+03,  6.0000e+00,  7.0000e+00,  ...

In [13]:
team_series_label

tensor([2, 2, 0,  ..., 1, 1, 0])

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FootballMatchPredictor(input_dim = 28, hidden_dim = 16, output_size=3)

In [15]:
def train_new(model, device, team_series_data, team_series_label):
    train_dataset = Data.TensorDataset(team_series_data, team_series_label)

    print(team_series_data.shape)
    print(team_series_label.shape)


    train_size = int(0.8 * len(train_dataset))  # 80% for training
    validation_size = len(train_dataset) - train_size  # 20% for validation
    train_dataset, validation_dataset = random_split(train_dataset, [train_size, validation_size])

    # train_dataset = CSVDataset(csv_file_data=args.train_data, csv_file_label=args.train_label)
    # train_loader = DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=True)
    train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=False)

    # # Load your dataset
    # train_dataset = CSVDataset(csv_file_data=args.train_data, csv_file_label='your_label_file.csv')  # Adjust the label file argument accordingly
    # train_loader = DataLoader(dataset=train_dataset, batch_size=args.batch_size, shuffle=True)
    
    # Define model, loss, and optimizer
    model.to(device)
    criterion = nn.CrossEntropyLoss() 
    optimizer = optim.Adam(model.parameters(), lr=0.00001)
    
    # Training loop
    for epoch in range(5):
        model.train()
        correct = 0
        total = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            # print(output.shape)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            # if batch_idx % 100 == 0:
            #     print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item() 

        training_accuracy = 100. * correct / total
        print(f'Epoch: {epoch} Training Accuracy: {training_accuracy:.2f}%')


        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for data, target in validation_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                val_total += target.size(0)
                val_correct += (predicted == target).sum().item()

        # Print validation accuracy for the epoch
        validation_accuracy = 100. * val_correct / val_total
        print(f'Epoch: {epoch} Validation Accuracy: {validation_accuracy:.2f}%')
            
    torch.save(model.state_dict(), "model_test.pth")
    print('Training Complete.')

In [16]:
train_new(model, device, team_series_data, team_series_label)

torch.Size([524440, 5, 28])
torch.Size([524440])
Epoch: 0 Training Accuracy: 46.21%
Epoch: 0 Validation Accuracy: 46.18%
Epoch: 1 Training Accuracy: 46.21%
Epoch: 1 Validation Accuracy: 46.18%
Epoch: 2 Training Accuracy: 46.21%
Epoch: 2 Validation Accuracy: 46.18%
Epoch: 3 Training Accuracy: 46.21%
Epoch: 3 Validation Accuracy: 46.18%
Epoch: 4 Training Accuracy: 46.21%
Epoch: 4 Validation Accuracy: 46.18%
Training Complete.
